# Results

## Table of Contents
* [1)*American Put Prices* Applying Least-Square Monte Carlo with Longstaff-Schwartz Parameters](#first-bullet)
    - [1a) Black-Scholes (*Akin to Table 1 in Longstaff-Schwartz*)](#firsta-bullet)
    - [1b) Jump Merton](#firstb-bullet)
    - [1c) Constant Elasticity of Variance (CEV)](#firstc-bullet)
* [2) *American Put Prices* Applying Least-Square Monte Carlo with Our Own Parameter Values](#second-bullet)
    - [2a) Black-Scholes](#seconda-bullet)
    - [2b) Jump Merton](#secondb-bullet)
    - [2c) Constant Elasticity of Variance (CEV)](#secondc-bullet)

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os

# local
from LSM import MonteCarloOptionPricing
from FD import AmericanPutFiniteDifference

## 1) *American Put Prices* Applying Least-Square Monte Carlo with Longstaff-Schwartz Parameters <a class="anchor" id="first-bullet"></a>
The simulation is based on 100,000 (50,000 plus 50,000 antithetic) paths for the stock price.

In [2]:
# Parameter inputs
r = 0.06  # risk-free rate
K = 40.   # strike price
λ = 0.4 # intensity rate of the Poisson process
dim = 100_000  # number of simulation paths
n = 50  # number of exercise rights per year
seed = 15001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

In [3]:
# Varying option parameters
S0_values = [36, 38, 40, 42, 44]
σ_values = [0.20, 0.40]
T_values = [1, 2]

### 1a) Black-Scholes (*Akin to Table 1 in Longstaff-Schwartz*) <a class="anchor" id="firsta-bullet"></a>


In [4]:
# Store results
Option_Pricing_Results_BS = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_BS = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Black-Scholes
            BS_Price = MC_Option_Pricing_BS.BS_option_value('put')
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_BS.GeometricBrownianMotion_vec() # simulate GBM paths
            American_put_price = MC_Option_Pricing_BS.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - BS_Price 
            
            # 4) Store the results
            Option_Pricing_Results_BS.append((S0, σ, T, BS_Price, American_put_price, Option_Premium))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, European Put: {BS_Price:.4f}, American Put Option: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}')

S0/Sigma/T: 36/0.2/1, European Put: 3.8443, American Put Option: 4.4720, Premium: 0.6277
S0/Sigma/T: 36/0.2/2, European Put: 3.7630, American Put Option: 4.8259, Premium: 1.0629
S0/Sigma/T: 36/0.4/1, European Put: 6.7114, American Put Option: 7.0751, Premium: 0.3637
S0/Sigma/T: 36/0.4/2, European Put: 7.7000, American Put Option: 8.4595, Premium: 0.7594
S0/Sigma/T: 38/0.2/1, European Put: 2.8519, American Put Option: 3.2400, Premium: 0.3881
S0/Sigma/T: 38/0.2/2, European Put: 2.9906, American Put Option: 3.7267, Premium: 0.7361
S0/Sigma/T: 38/0.4/1, European Put: 5.8343, American Put Option: 6.1222, Premium: 0.2879
S0/Sigma/T: 38/0.4/2, European Put: 6.9788, American Put Option: 7.6295, Premium: 0.6507
S0/Sigma/T: 40/0.2/1, European Put: 2.0664, American Put Option: 2.3049, Premium: 0.2385
S0/Sigma/T: 40/0.2/2, European Put: 2.3559, American Put Option: 2.8727, Premium: 0.5168
S0/Sigma/T: 40/0.4/1, European Put: 5.0596, American Put Option: 5.2752, Premium: 0.2155
S0/Sigma/T: 40/0.4/2,

### 1b) Jump Merton <a class="anchor" id="firstb-bullet"></a>

In [5]:
λ = 0.4 # intensity rate of the Poisson process
α = -0.5 # Mean of log-normal jump size
β = 0.4 # Volatility of log-normal jump size

In [6]:
# Store results
Option_Pricing_Results_JM = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_JM = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Jump Merton semi-closed solution
            JumpMerton_price = MC_Option_Pricing_JM.merton_jump_option_value(α, β)
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_JM.MertonJumpDiffusion_vec(α, β) # simulate jump paths
            American_put_price = MC_Option_Pricing_JM.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - JumpMerton_price 
            
            # 4) Store the results
            Option_Pricing_Results_JM.append((S0, σ, T, JumpMerton_price, American_put_price, Option_Premium))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, European Put: {JumpMerton_price:.4f}, American Put Option: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}')

S0/Sigma/T: 36/0.2/1, European Put: 5.7636, American Put Option: 6.0735, Premium: 0.3099
S0/Sigma/T: 36/0.2/2, European Put: 7.1273, American Put Option: 8.0148, Premium: 0.8875
S0/Sigma/T: 36/0.4/1, European Put: 8.0853, American Put Option: 8.3795, Premium: 0.2942
S0/Sigma/T: 36/0.4/2, European Put: 9.8050, American Put Option: 10.6486, Premium: 0.8435
S0/Sigma/T: 38/0.2/1, European Put: 5.0684, American Put Option: 5.2855, Premium: 0.2171
S0/Sigma/T: 38/0.2/2, European Put: 6.5609, American Put Option: 7.3615, Premium: 0.8005
S0/Sigma/T: 38/0.4/1, European Put: 7.3232, American Put Option: 7.5697, Premium: 0.2464
S0/Sigma/T: 38/0.4/2, European Put: 9.1839, American Put Option: 9.9464, Premium: 0.7625
S0/Sigma/T: 40/0.2/1, European Put: 4.5068, American Put Option: 4.6781, Premium: 0.1713
S0/Sigma/T: 40/0.2/2, European Put: 6.0619, American Put Option: 6.7859, Premium: 0.7241
S0/Sigma/T: 40/0.4/1, European Put: 6.6461, American Put Option: 6.8395, Premium: 0.1934
S0/Sigma/T: 40/0.4/2

### 1c) Constant Elasticity of Variance (CEV) <a class="anchor" id="firstc-bullet"></a>

In [7]:
γ = 1.5 # parameter governing elasticity with respect to price, if set = 2 then we obtain BS prices

In [8]:
# Store results
Option_Pricing_Results_CEV = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_CEV = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Compute American Put using LSM
            MC_Option_Pricing_CEV.CEV(γ) # simulate CEV paths
            American_put_price = MC_Option_Pricing_CEV.american_option_LSM(poly_degree, otype='put') # price option
            
            # 2) Store the results
            Option_Pricing_Results_CEV.append((S0, σ, T, American_put_price))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, American Put Option: {American_put_price[0]:.4f}')

S0/Sigma/T: 36/0.2/1, American Put Option: 3.9521
S0/Sigma/T: 36/0.2/2, American Put Option: 3.9042
S0/Sigma/T: 36/0.4/1, American Put Option: 4.0976
S0/Sigma/T: 36/0.4/2, American Put Option: 4.2488
S0/Sigma/T: 38/0.2/1, American Put Option: 1.9581
S0/Sigma/T: 38/0.2/2, American Put Option: 1.9365
S0/Sigma/T: 38/0.4/1, American Put Option: 2.6965
S0/Sigma/T: 38/0.4/2, American Put Option: 2.9877
S0/Sigma/T: 40/0.2/1, American Put Option: 0.6200
S0/Sigma/T: 40/0.2/2, American Put Option: 0.6864
S0/Sigma/T: 40/0.4/1, American Put Option: 1.7071
S0/Sigma/T: 40/0.4/2, American Put Option: 2.0702
S0/Sigma/T: 42/0.2/1, American Put Option: 0.1616
S0/Sigma/T: 42/0.2/2, American Put Option: 0.2268
S0/Sigma/T: 42/0.4/1, American Put Option: 1.0298
S0/Sigma/T: 42/0.4/2, American Put Option: 1.4140
S0/Sigma/T: 44/0.2/1, American Put Option: 0.0356
S0/Sigma/T: 44/0.2/2, American Put Option: 0.0714
S0/Sigma/T: 44/0.4/1, American Put Option: 0.5994
S0/Sigma/T: 44/0.4/2, American Put Option: 0.9461


## 2) *American Put Prices* Applying Least-Square Monte Carlo with Our Own Parameter Values <a class="anchor" id="second-bullet"></a>
The simulation is based on 100,000 (50,000 plus 50,000 antithetic) paths for the stock price.

In [9]:
# Parameter inputs
r = 0.04  # risk-free rate
K = 100.   # strike price
λ = 0.4 # intensity rate of the Poisson process
dim = 100_000  # number of simulation paths
n = 50  # number of exercise rights per year
seed = 15001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

In [10]:
# Option parameters
S0_values = [90, 95, 100, 105, 110]
σ_values = [0.20, 0.40]
T_values = [1, 2]

## 2a) Black-Scholes <a class="anchor" id="seconda-bullet"></a>

In [11]:
# Store results
Results_BS = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_BS = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Black-Scholes
            BS_Price = MC_Option_Pricing_BS.BS_option_value('put')
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_BS.GeometricBrownianMotion_vec() # simulate GBM paths
            American_put_price = MC_Option_Pricing_BS.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - BS_Price 
            
            # 4) Store the results
            Results_BS.append((S0, σ, T, BS_Price, American_put_price, Option_Premium))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, European Put: {BS_Price:.4f}, American Put Option: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}')

S0/Sigma/T: 90/0.2/1, European Put: 10.8414, American Put Option: 11.7552, Premium: 0.9138
S0/Sigma/T: 90/0.2/2, European Put: 11.4484, American Put Option: 13.0895, Premium: 1.6411
S0/Sigma/T: 90/0.4/1, European Put: 17.9818, American Put Option: 18.4854, Premium: 0.5036
S0/Sigma/T: 90/0.4/2, European Put: 21.4763, American Put Option: 22.6126, Premium: 1.1363
S0/Sigma/T: 95/0.2/1, European Put: 8.1618, American Put Option: 8.7427, Premium: 0.5809
S0/Sigma/T: 95/0.2/2, European Put: 9.2425, American Put Option: 10.3973, Premium: 1.1548
S0/Sigma/T: 95/0.4/1, European Put: 15.6903, American Put Option: 16.0652, Premium: 0.3749
S0/Sigma/T: 95/0.4/2, European Put: 19.5394, American Put Option: 20.5113, Premium: 0.9720
S0/Sigma/T: 100/0.2/1, European Put: 6.0040, American Put Option: 6.3504, Premium: 0.3464
S0/Sigma/T: 100/0.2/2, European Put: 7.3963, American Put Option: 8.2211, Premium: 0.8248
S0/Sigma/T: 100/0.4/1, European Put: 13.6572, American Put Option: 13.9399, Premium: 0.2827
S0/

## 2b) Jump Merton <a class="anchor" id="secondb-bullet"></a>

In [12]:
λ = 0.4 # intensity rate of the Poisson process
α = -0.5 # Mean of log-normal jump size
β = 0.4 # Volatility of log-normal jump size

In [13]:
# Store results
Results_JM = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_JM = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Jump Merton semi-closed solution
            JumpMerton_price = MC_Option_Pricing_JM.merton_jump_option_value(α, β)
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_JM.MertonJumpDiffusion_vec(α, β) # simulate jump paths
            American_put_price = MC_Option_Pricing_JM.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - JumpMerton_price 
            
            # 4) Store the results
            Results_JM.append((S0, σ, T, JumpMerton_price, American_put_price, Option_Premium))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, European Put: {JumpMerton_price:.4f}, American Put Option: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}')

S0/Sigma/T: 90/0.2/1, European Put: 15.4293, American Put Option: 15.7966, Premium: 0.3673
S0/Sigma/T: 90/0.2/2, European Put: 19.7099, American Put Option: 21.1163, Premium: 1.4064
S0/Sigma/T: 90/0.4/1, European Put: 21.3713, American Put Option: 21.7120, Premium: 0.3407
S0/Sigma/T: 90/0.4/2, European Put: 26.7443, American Put Option: 28.0151, Premium: 1.2708
S0/Sigma/T: 95/0.2/1, European Put: 13.5498, American Put Option: 13.7760, Premium: 0.2262
S0/Sigma/T: 95/0.2/2, European Put: 18.1511, American Put Option: 19.4070, Premium: 1.2558
S0/Sigma/T: 95/0.4/1, European Put: 19.3825, American Put Option: 19.6271, Premium: 0.2446
S0/Sigma/T: 95/0.4/2, European Put: 25.0869, American Put Option: 26.2253, Premium: 1.1384
S0/Sigma/T: 100/0.2/1, European Put: 12.0261, American Put Option: 12.1877, Premium: 0.1616
S0/Sigma/T: 100/0.2/2, European Put: 16.7778, American Put Option: 17.8976, Premium: 1.1198
S0/Sigma/T: 100/0.4/1, European Put: 17.6104, American Put Option: 17.7853, Premium: 0.1

## 2c) Constant Elasticity of Variance (CEV) <a class="anchor" id="secondc-bullet"></a>

In [14]:
γ = 1.5 # parameter governing elasticity with respect to price, if set = 2 then we obtain BS prices

In [ ]:
# Store results
Results_CEV = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_CEV = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Compute American Put using LSM
            MC_Option_Pricing_CEV.CEV(γ) # simulate CEV paths
            American_put_price = MC_Option_Pricing_CEV.american_option_LSM(poly_degree, otype='put') # price option
            
            # 2) Store the results
            Results_CEV.append((S0, σ, T, American_put_price))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, American Put Option: {American_put_price[0]:.4f}')

S0/Sigma/T: 90/0.2/1, American Put Option: 9.9201
S0/Sigma/T: 90/0.2/2, American Put Option: 9.8402
S0/Sigma/T: 90/0.4/1, American Put Option: 10.0503
S0/Sigma/T: 90/0.4/2, American Put Option: 10.2995
S0/Sigma/T: 95/0.2/1, American Put Option: 4.9222
S0/Sigma/T: 95/0.2/2, American Put Option: 4.8630
S0/Sigma/T: 95/0.4/1, American Put Option: 6.2244
S0/Sigma/T: 95/0.4/2, American Put Option: 6.8937
S0/Sigma/T: 100/0.2/1, American Put Option: 1.3677
S0/Sigma/T: 100/0.2/2, American Put Option: 1.5591
S0/Sigma/T: 100/0.4/1, American Put Option: 3.6065
S0/Sigma/T: 100/0.4/2, American Put Option: 4.4842
